In [1]:
import numpy as np
import math

import matplotlib.pyplot as plt
import cartopy.crs as crs
import cartopy.feature as cfeature
import matplotlib.colors as colors
from matplotlib.cm import get_cmap
from matplotlib import ticker
import matplotlib.gridspec as gridspec

from cartopy import config
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.util import add_cyclic_point

from IPython.display import Image

from tqdm import tqdm
import os 
from datetime import date

g = 9.8

In [2]:
Year = "2000"

Months = []
for imon in range(1,2): ### shw
    Months.append(str(imon).zfill(2))
    
Days = []
for iday in range(7,26,3):
    Days.append(str(iday).zfill(2))
    
print(Year)
print(Months)
print(Days)

2000
['01']
['07', '10', '13', '16', '19', '22', '25']


In [3]:
directory = '/n/home12/hongwei/HONGWEI/lagranto_era5_0.2um/Simulation_0.2um/'+Year+'_01/'


N_head = 5 # first 5 lines are head lines, not include data
N_column = 4+2

Nx = 36
Ny = 21
Nz = 5
N_parcel = 3780

if Nx*Ny*Nz!=N_parcel: print('ERROR: parcel number is wrong!')
    
    
# filename = "traj_"+Year+Months[0]+Days[0]+"_trace.1"
# file1 = open(directory+filename, 'r')
# Lines = file1.readlines()
# print(Nt,"days")


# for DT/DZ compare:
Criterion = 0.0

In [4]:
Lats_edge = np.arange(-90,91,6)
Lons_edge = np.arange(-180,181,10)

Lats_mid = np.arange(-87,90,6)
Lons_mid = np.arange(-175,180,10)
Levs = [100, 75, 55, 40, 30]


N_lat = len(Lats_mid)
N_lon = len(Lons_mid)
N_lev = 5 # 16, 18, 20, 22, 24 km

Lats_edge, Lons_edge, Lats_mid, Lons_mid, N_lat, N_lon, Lons_edge[-1]

(array([-90, -84, -78, -72, -66, -60, -54, -48, -42, -36, -30, -24, -18,
        -12,  -6,   0,   6,  12,  18,  24,  30,  36,  42,  48,  54,  60,
         66,  72,  78,  84,  90]),
 array([-180, -170, -160, -150, -140, -130, -120, -110, -100,  -90,  -80,
         -70,  -60,  -50,  -40,  -30,  -20,  -10,    0,   10,   20,   30,
          40,   50,   60,   70,   80,   90,  100,  110,  120,  130,  140,
         150,  160,  170,  180]),
 array([-87, -81, -75, -69, -63, -57, -51, -45, -39, -33, -27, -21, -15,
         -9,  -3,   3,   9,  15,  21,  27,  33,  39,  45,  51,  57,  63,
         69,  75,  81,  87]),
 array([-175, -165, -155, -145, -135, -125, -115, -105,  -95,  -85,  -75,
         -65,  -55,  -45,  -35,  -25,  -15,   -5,    5,   15,   25,   35,
          45,   55,   65,   75,   85,   95,  105,  115,  125,  135,  145,
         155,  165,  175]),
 30,
 36,
 180)

In [5]:

def find_i_LON(lon):
    if lon<Lons_edge[0]:  lon = lon+360
    if lon>=Lons_edge[-1]: lon = lon-360
    return int( np.floor( (lon - Lons_edge[0]) / (Lons_edge[1]-Lons_edge[0]) ) )


def find_i_LAT(lat):
    return int( np.floor( (lat - Lats_edge[0]) / (Lats_edge[1]-Lats_edge[0]) ) )


def find_i_LEV(lev):
    if lev==Levs[0]: i=0
    if lev==Levs[1]: i=1
    if lev==Levs[2]: i=2
    if lev==Levs[3]: i=3
    if lev==Levs[4]: i=4
    return i


In [6]:
# loop for all traj files

# count the particle (injected at different height) number in each grid cell
N_day = 20*366
Num_2D = np.zeros((N_lon, N_lat, N_lev, N_day)) 
        
for month in Months:
    for day in tqdm(Days):
        
        
        f_date = date(2000, 1, 1)
        l_date = date(int(Year), int(month), int(day))
        Delta  = l_date - f_date
        i_day = Delta.days

        
        # (1) read original data from traj files
        filename = "traj_"+Year+month+day+"_trace.1"
        file1 = open(directory+filename, 'r')
        Lines = file1.readlines()

        Nt = int( (len(Lines)-4)/N_parcel - 1 )   
        data = np.zeros((N_parcel, Nt, N_column))
        
        
        count = 0
        # Strips the newline character
        for line in Lines:
            count += 1
    
            if count>=5:
                i = count-5
                i_parcel = math.floor( i / (Nt+1) ) # Nt time lines plue 1 empty line
                i_t = i%(Nt+1)
        
                if i_t!=0:
                    a = line.split()

                    if i_t==1:
                        if float(a[0])!=0.0: print('ERROR: first time is not 0 !!!')
                
                    data[i_parcel,i_t-1,0] = float(a[0]) # t [h]
                    data[i_parcel,i_t-1,1] = float(a[1]) # lon
                    data[i_parcel,i_t-1,2] = float(a[2]) # lat
                    data[i_parcel,i_t-1,3] = float(a[3]) # lev
                    data[i_parcel,i_t-1,4] = float(a[4]) # RHO
                    data[i_parcel,i_t-1,5] = float(a[5]) # DTDP
                
                
        # (2) calculate injected tracer lifetime in the stratosphere
        
        for i_parcel in range(N_parcel):
            In_strat = 1
            for it in range(Nt-4):
                    
                RHO_1 = data[i_parcel,it,4]   # [kg/m3]
                RHO_2 = data[i_parcel,it+1,4] # 
                RHO_3 = data[i_parcel,it+2,4] # 
                RHO_4 = data[i_parcel,it+3,4] # 
                RHO_5 = data[i_parcel,it+4,4] # 
                    
                DTDP_1 = data[i_parcel,it,5]   # [K/Pa]
                DTDP_2 = data[i_parcel,it+1,5] # [K/Pa]
                DTDP_3 = data[i_parcel,it+2,5] # [K/Pa]
                DTDP_4 = data[i_parcel,it+3,5] # [K/Pa]
                DTDP_5 = data[i_parcel,it+4,5] # [K/Pa]
                    
                DTDZ_1 = -1*DTDP_1 *g*RHO_1 * 1000.0 # [K/km]
                DTDZ_2 = -1*DTDP_2 *g*RHO_2 * 1000.0
                DTDZ_3 = -1*DTDP_3 *g*RHO_3 * 1000.0
                DTDZ_4 = -1*DTDP_4 *g*RHO_4 * 1000.0
                DTDZ_5 = -1*DTDP_5 *g*RHO_5 * 1000.0
                    
                P_1 = data[i_parcel,it,3]
                P_2 = data[i_parcel,it+1,3]
                P_3 = data[i_parcel,it+2,3]
                    
                if P_1>400.0: # directly identified as tropopause
                    In_strat = 0
                    break 
                            
                elif P_1>70.0: # the particle is lower than 70 hPa level
                    if DTDZ_1<Criterion and DTDZ_2<Criterion and DTDZ_3<Criterion \
                                        and DTDZ_4<Criterion and DTDZ_5<Criterion: 
                        if P_1<P_2 and P_2<P_3: # keep dropping down
                            In_strat = 0                            
                            break 
                    
                # count number of particle in the stratosphere in each grid cell
                if In_strat ==1:
                    LON = data[i_parcel,it,1]
                    LAT = data[i_parcel,it,2]
                    LEV = data[i_parcel,0,3] # consider the initial injection height
                        
                    i_LON = find_i_LON(LON)
                    i_LAT = find_i_LAT(LAT)
                    i_LEV = find_i_LEV(LEV)
                          
#                     print(LON, LAT, LEV, i_LON, i_LAT, i_LEV, it)
                    Num_2D[i_LON, i_LAT, i_LEV, i_day+it] = Num_2D[i_LON, i_LAT, i_LEV, i_day+it] + 1
    
        file1.close()
        

100%|██████████| 7/7 [09:07<00:00, 78.18s/it]


In [7]:
        
with open('./Num_Concnt_1st/Num_Concnt_2000_'+Year+'.txt', 'w') as f:
    for it in range(N_day):
            for ix in range(N_lon):
                for iy in range(N_lat):
                    f.write(  str(it) + ',' \
                            + str(Lons_mid[ix]) + ',' \
                            + str(Lats_mid[iy]) + ',' \
                        
                            + str(Num_2D[ix, iy, 0, it])  + ',' \
                            + str(Num_2D[ix, iy, 1, it])  + ',' \
                            + str(Num_2D[ix, iy, 2, it])  + ',' \
                            + str(Num_2D[ix, iy, 3, it])  + ',' \
                            + str(Num_2D[ix, iy, 4, it])   )
                        
                    f.write('\n')
f.close()

In [8]:
Nt

3654